### Kaggle TensorFlow Speech Competition
Use TensorFlow Simple Audio Recognition tutorial code to create submitssion file.

In [1]:
# download dataset
import os, sys

NOTEBOOK_DIR = '/notebooks'
DATA_DIR = '/notebooks/data'
TRAIN_DIR = DATA_DIR + '/training_atom_xavier_batchnorm'
TEST_DIR = DATA_DIR + "/test/audio"
KAGGLE_TRAIN_DATA_DIR = DATA_DIR + '/train/audio'

if os.path.isdir(DATA_DIR):
    os.chdir(DATA_DIR)
else:
    os.chdir(NOTEBOOK_DIR)
    os.mkdir('data')
    os.chdir('data')

print (os.getcwd())

# Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# assume that the data is downloaded and extracted

/notebooks/data


In [2]:
# select some files as samples
#os.mkdir('sample/train')
#os.mkdir('sample/valid')
#os.mkdir('sample/result')

### Try speech_command sample

In [3]:
import argparse

import numpy as np
from six.moves import xrange
import tensorflow as tf

import input_data; reload (input_data)
import models
from tensorflow.python.platform import gfile

# Set up argument list
parser = argparse.ArgumentParser()
parser.add_argument('--data_url', type=str,
                    # pylint: disable=line-too-long
                    #default='http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                    default='',
                    # pylint: enable=line-too-long
                    help='Location of speech training data archive on the web.')
parser.add_argument('--data_dir', type=str, default=KAGGLE_TRAIN_DATA_DIR,
                    help="""\
                    Where to download the speech training data to.
                    """)
parser.add_argument('--background_volume', type=float, default=0.1,
                    help="""\
                    How loud the background noise should be, between 0 and 1.
                    """)
parser.add_argument('--background_frequency', type=float, default=0.8,
                    help="""\
                    How many of the training samples have background noise mixed in.
                    """)
parser.add_argument('--silence_percentage', type=float, default=10.0,
                    help="""\
                    How much of the training data should be silence.
                    """)
parser.add_argument('--unknown_percentage', type=float, default=10.0,
                    help="""\
                    How much of the training data should be unknown words.
                    """)
parser.add_argument('--time_shift_ms', type=float, default=100.0,
                    help="""\
                    Range to randomly shift the training audio by in time.
                    """)
parser.add_argument('--testing_percentage', type=int, default=10,
                    help='What percentage of wavs to use as a test set.')
parser.add_argument('--validation_percentage', type=int, default=10,
                    help='What percentage of wavs to use as a validation set.')
parser.add_argument('--sample_rate', type=int, default=16000,
                    help='Expected sample rate of the wavs',)
parser.add_argument('--clip_duration_ms', type=int, default=1000,
                    help='Expected duration in milliseconds of the wavs',)
parser.add_argument('--window_size_ms', type=float, default=30.0,
                    help='How long each spectrogram timeslice is',)
parser.add_argument('--window_stride_ms', type=float, default=10.0,
                    help='How long each spectrogram timeslice is',)
parser.add_argument('--dct_coefficient_count', type=int, default=40,
                    help='How many bins to use for the MFCC fingerprint',)
# default='15000,3000',
parser.add_argument('--how_many_training_steps', type=str, default='1500,1000,1000',
                    help='How many training loops to run',)
parser.add_argument('--eval_step_interval', type=int, default=400,
                    help='How often to evaluate the training results.')
parser.add_argument('--learning_rate', type=str, default='0.001,0.0001,0.00001',
                    help='How large a learning rate to use when training.')
parser.add_argument('--batch_size', type=int, default=100,
                    help='How many items to train with at once',)
parser.add_argument('--summaries_dir', type=str, default=TRAIN_DIR + '/retrain_logs',
                    help='Where to save summary logs for TensorBoard.')
parser.add_argument('--wanted_words', type=str, default='yes,no,up,down,left,right,on,off,stop,go',
                    help='Words to use (others will be added to an unknown label)',)
parser.add_argument('--train_dir', type=str, default=TRAIN_DIR + '/speech_commands_train',
                    help='Directory to write event logs and checkpoint.')
parser.add_argument('--save_step_interval', type=int, default=100,
                    help='Save model checkpoint every save_steps.')
#--start_checkpoint=/tmp/speech_commands_train/conv.ckpt-100 or TRAIN_DIR + '/speech_commands_train/conv.ckpt-2400'
parser.add_argument('--start_checkpoint', type=str, default='',
                    help='If specified, restore this pretrained model before any training.')
parser.add_argument('--model_architecture', type=str, default='conv',
                    help='What model architecture to use')
parser.add_argument('--check_nans', type=bool, default=False,
                    help='Whether to check for invalid numbers during processing')

FLAGS, unparsed = parser.parse_known_args()

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

# Start a new TensorFlow session.
sess = tf.InteractiveSession()

# Begin by making sure we have the training data we need. If you already have
# training data of your own, use `--data_url= ` on the command line to avoid
# downloading.
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
    FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
    FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
    FLAGS.data_url, FLAGS.data_dir, FLAGS.silence_percentage,
    FLAGS.unknown_percentage,
    FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
    FLAGS.testing_percentage, model_settings)
fingerprint_size = model_settings['fingerprint_size']
label_count = model_settings['label_count']
time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)

Words to work with :
{'sheila': True, 'seven': True, 'right': True, 'house': True, 'up': True, 'down': True, 'zero': True, 'go': True, 'yes': True, 'no': True, 'wow': True, 'six': True, 'two': True, 'bird': True, 'happy': True, 'marvin': True, 'stop': True, 'five': True, 'one': True, 'on': True, 'off': True, 'four': True, 'tree': True, 'dog': True, 'bed': True, 'cat': True, 'nine': True, 'three': True, 'eight': True, 'left': True}
Words in interest :
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [5]:
print (model_settings)
print (audio_processor)
print (fingerprint_size)
print (label_count)
print (time_shift_samples)
print (input_data.prepare_words_list(FLAGS.wanted_words.split(',')))

{'fingerprint_size': 3920, 'dct_coefficient_count': 40, 'sample_rate': 16000, 'window_size_samples': 480, 'spectrogram_length': 98, 'desired_samples': 16000, 'window_stride_samples': 160, 'label_count': 12}
3920
12
1600
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [6]:
# Figure out the learning rates for each training phase. Since it's often
# effective to have high learning rates at the start of training, followed by
# lower levels towards the end, the number of steps and learning rates can be
# specified as comma-separated lists to define the rate at each stage. For
# example --how_many_training_steps=10000,3000 --learning_rate=0.001,0.0001
# will run 13,000 training loops in total, with a rate of 0.001 for the first
# 10,000, and 0.0001 for the final 3,000.
training_steps_list = list(map(int, FLAGS.how_many_training_steps.split(',')))
learning_rates_list = list(map(float, FLAGS.learning_rate.split(',')))
if len(training_steps_list) != len(learning_rates_list):
    raise Exception(
        '--how_many_training_steps and --learning_rate must be equal length '
        'lists, but are %d and %d long instead' % (len(training_steps_list),
                                                   len(learning_rates_list)))

In [7]:
print (training_steps_list)
print (learning_rates_list)
print (FLAGS.model_architecture)

[1500, 1000, 1000]
[0.001, 0.0001, 1e-05]
conv


In [8]:
fingerprint_input = tf.placeholder(tf.float32, [None, fingerprint_size], name='fingerprint_input')
logits, dropout_prob = models.create_model(fingerprint_input, model_settings,
                                           FLAGS.model_architecture, is_training=True)
# Define loss and optimizer
ground_truth_input = tf.placeholder(tf.float32, [None, label_count], name='groundtruth_input')

# Optionally we can add runtime checks to spot when NaNs or other symptoms of
# numerical errors start occurring during training.
control_dependencies = []
if FLAGS.check_nans:
    checks = tf.add_check_numerics_ops()
    control_dependencies = [checks]

In [9]:
# Create the back propagation and training evaluation machinery in the graph.
with tf.name_scope('cross_entropy'):
    cross_entropy_mean = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_input, logits=logits))
tf.summary.scalar('cross_entropy', cross_entropy_mean)
    
with tf.name_scope('train'), tf.control_dependencies(control_dependencies):
    learning_rate_input = tf.placeholder(
        tf.float32, [], name='learning_rate_input')
    train_step = tf.train.AdamOptimizer(
        learning_rate_input).minimize(cross_entropy_mean)
    
predicted_indices = tf.argmax(logits, 1)
expected_indices = tf.argmax(ground_truth_input, 1)
correct_prediction = tf.equal(predicted_indices, expected_indices)
confusion_matrix = tf.confusion_matrix(expected_indices, predicted_indices)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', evaluation_step)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [10]:
global_step = tf.contrib.framework.get_or_create_global_step()
increment_global_step = tf.assign(global_step, global_step + 1)

saver = tf.train.Saver(tf.global_variables())

# Merge all the summaries and write them out to /tmp/retrain_logs (by default)
merged_summaries = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                   sess.graph)
validation_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/validation')

Instructions for updating:
Please switch to tf.train.get_or_create_global_step


In [11]:
tf.global_variables_initializer().run()

start_step = 1

if FLAGS.start_checkpoint:
    models.load_variables_from_checkpoint(sess, FLAGS.start_checkpoint)
    start_step = global_step.eval(session=sess)

tf.logging.info('Training from step: %d ', start_step)

# Save graph.pbtxt.
tf.train.write_graph(sess.graph_def, FLAGS.train_dir,
                     FLAGS.model_architecture + '.pbtxt')

# Save list of words.
with gfile.GFile(
    os.path.join(FLAGS.train_dir, FLAGS.model_architecture + '_labels.txt'),
    'w') as f:
    f.write('\n'.join(audio_processor.words_list)+'\n')
    
print ("Word list : ")
print (audio_processor.words_list)

INFO:tensorflow:Training from step: 1 
Word list : 
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [12]:
# Training loop.
training_steps_max = np.sum(training_steps_list)
for training_step in xrange(start_step, training_steps_max + 1):
    # Figure out what the current learning rate is.
    training_steps_sum = 0
    for i in range(len(training_steps_list)):
        training_steps_sum += training_steps_list[i]
        if training_step <= training_steps_sum:
            learning_rate_value = learning_rates_list[i]
            break
    # Pull the audio samples we'll use for training.
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        FLAGS.batch_size, 0, model_settings, FLAGS.background_frequency,
        FLAGS.background_volume, time_shift_samples, 'training', sess)
    # Run the graph with this batch of training data.
    train_summary, train_accuracy, cross_entropy_value, _, _ = sess.run(
        [
            merged_summaries, evaluation_step, cross_entropy_mean, train_step,
            increment_global_step
        ],
        feed_dict={
            fingerprint_input: train_fingerprints,
            ground_truth_input: train_ground_truth,
            learning_rate_input: learning_rate_value,
            dropout_prob: 0.5
        })
    train_writer.add_summary(train_summary, training_step)
    if (training_step % 10 == 0):
        tf.logging.info('Step #%d: rate %f, accuracy %.1f%%, cross entropy %f' %
                        (training_step, learning_rate_value, train_accuracy * 100,
                         cross_entropy_value))
    is_last_step = (training_step == training_steps_max)
    if (training_step % FLAGS.eval_step_interval) == 0 or is_last_step:
      set_size = audio_processor.set_size('validation')
      total_accuracy = 0
      total_conf_matrix = None
      for i in xrange(0, set_size, FLAGS.batch_size):
        validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(FLAGS.batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy, conf_matrix = sess.run(
            [merged_summaries, evaluation_step, confusion_matrix],
            feed_dict={
                fingerprint_input: validation_fingerprints,
                ground_truth_input: validation_ground_truth,
                dropout_prob: 1.0
            })
        validation_writer.add_summary(validation_summary, training_step)
        batch_size = min(FLAGS.batch_size, set_size - i)
        total_accuracy += (validation_accuracy * batch_size) / set_size
        if total_conf_matrix is None:
          total_conf_matrix = conf_matrix
        else:
          total_conf_matrix += conf_matrix
      tf.logging.info('Confusion Matrix:\n %s' % (total_conf_matrix))
      tf.logging.info('Step %d: Validation accuracy = %.1f%% (N=%d)' %
                      (training_step, total_accuracy * 100, set_size))

    # Save the model checkpoint periodically.
    if (training_step % FLAGS.save_step_interval == 0 or
        training_step == training_steps_max):
      checkpoint_path = os.path.join(FLAGS.train_dir,
                                     FLAGS.model_architecture + '.ckpt')
      tf.logging.info('Saving to "%s-%d"', checkpoint_path, training_step)
      saver.save(sess, checkpoint_path, global_step=training_step)


INFO:tensorflow:Step #10: rate 0.001000, accuracy 12.0%, cross entropy 4.398624
INFO:tensorflow:Step #20: rate 0.001000, accuracy 14.0%, cross entropy 3.163670
INFO:tensorflow:Step #30: rate 0.001000, accuracy 14.0%, cross entropy 2.631719
INFO:tensorflow:Step #40: rate 0.001000, accuracy 26.0%, cross entropy 2.640296
INFO:tensorflow:Step #50: rate 0.001000, accuracy 14.0%, cross entropy 2.532232
INFO:tensorflow:Step #60: rate 0.001000, accuracy 22.0%, cross entropy 2.855088
INFO:tensorflow:Step #70: rate 0.001000, accuracy 21.0%, cross entropy 2.455455
INFO:tensorflow:Step #80: rate 0.001000, accuracy 24.0%, cross entropy 2.924994
INFO:tensorflow:Step #90: rate 0.001000, accuracy 20.0%, cross entropy 2.512482
INFO:tensorflow:Step #100: rate 0.001000, accuracy 36.0%, cross entropy 2.528648
INFO:tensorflow:Saving to "/notebooks/data/training_atom_xavier_batchnorm/speech_commands_train/conv.ckpt-100"
INFO:tensorflow:Step #110: rate 0.001000, accuracy 33.0%, cross entropy 2.014298
INFO:te

INFO:tensorflow:Step 800: Validation accuracy = 78.9% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_atom_xavier_batchnorm/speech_commands_train/conv.ckpt-800"
INFO:tensorflow:Step #810: rate 0.001000, accuracy 80.0%, cross entropy 0.683768
INFO:tensorflow:Step #820: rate 0.001000, accuracy 86.0%, cross entropy 0.505517
INFO:tensorflow:Step #830: rate 0.001000, accuracy 87.0%, cross entropy 0.481175
INFO:tensorflow:Step #840: rate 0.001000, accuracy 74.0%, cross entropy 0.800742
INFO:tensorflow:Step #850: rate 0.001000, accuracy 84.0%, cross entropy 0.585398
INFO:tensorflow:Step #860: rate 0.001000, accuracy 76.0%, cross entropy 0.718785
INFO:tensorflow:Step #870: rate 0.001000, accuracy 81.0%, cross entropy 0.575931
INFO:tensorflow:Step #880: rate 0.001000, accuracy 71.0%, cross entropy 0.785514
INFO:tensorflow:Step #890: rate 0.001000, accuracy 82.0%, cross entropy 0.505459
INFO:tensorflow:Step #900: rate 0.001000, accuracy 81.0%, cross entropy 0.521643
INFO:tensorflow:

INFO:tensorflow:Confusion Matrix:
 [[258   0   0   0   0   0   0   0   0   0   0   0]
 [  0 175   1   9   2   6   2   2  43   4   0  14]
 [  1   6 238  13   0   0   0   0   1   0   0   2]
 [  0   0   1 248   0   3   0   0   0   0   0  18]
 [  0   7   0   0 201   0   0   0   4  36   2  10]
 [  0   5   0  23   0 213   0   0   7   2   0  14]
 [  0  13  11  12   3   0 199   2   1   3   0   3]
 [  0  20   1   2   0   0   0 224   6   3   0   0]
 [  1   2   0   0   0   0   0   0 249   2   0   3]
 [  0   3   0   0   1   0   0   0  13 238   0   1]
 [  1  11   1   1  17   0   0   0   7   8 186  14]
 [  2   4   0  24   0   1   0   0   9   3   0 217]]
INFO:tensorflow:Step 1600: Validation accuracy = 85.5% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_atom_xavier_batchnorm/speech_commands_train/conv.ckpt-1600"
INFO:tensorflow:Step #1610: rate 0.000100, accuracy 86.0%, cross entropy 0.396055
INFO:tensorflow:Step #1620: rate 0.000100, accuracy 89.0%, cross entropy 0.334318
INFO:tensorf

INFO:tensorflow:Step #2330: rate 0.000100, accuracy 88.0%, cross entropy 0.454610
INFO:tensorflow:Step #2340: rate 0.000100, accuracy 93.0%, cross entropy 0.188416
INFO:tensorflow:Step #2350: rate 0.000100, accuracy 91.0%, cross entropy 0.340004
INFO:tensorflow:Step #2360: rate 0.000100, accuracy 96.0%, cross entropy 0.164329
INFO:tensorflow:Step #2370: rate 0.000100, accuracy 81.0%, cross entropy 0.502679
INFO:tensorflow:Step #2380: rate 0.000100, accuracy 86.0%, cross entropy 0.438345
INFO:tensorflow:Step #2390: rate 0.000100, accuracy 92.0%, cross entropy 0.275590
INFO:tensorflow:Step #2400: rate 0.000100, accuracy 91.0%, cross entropy 0.314040
INFO:tensorflow:Confusion Matrix:
 [[258   0   0   0   0   0   0   0   0   0   0   0]
 [  0 186   3   8   1   7   2   2  27   8   0  14]
 [  0   8 247   3   0   0   2   0   1   0   0   0]
 [  0   5   2 237   1   2   1   0   0   0   0  22]
 [  0   8   1   0 211   0   0   0   5  31   0   4]
 [  0   6   1  18   0 214   0   0   6   2   1  16]
 [ 

INFO:tensorflow:Step #3060: rate 0.000010, accuracy 90.0%, cross entropy 0.270862
INFO:tensorflow:Step #3070: rate 0.000010, accuracy 89.0%, cross entropy 0.348674
INFO:tensorflow:Step #3080: rate 0.000010, accuracy 90.0%, cross entropy 0.372427
INFO:tensorflow:Step #3090: rate 0.000010, accuracy 93.0%, cross entropy 0.281238
INFO:tensorflow:Step #3100: rate 0.000010, accuracy 89.0%, cross entropy 0.349735
INFO:tensorflow:Saving to "/notebooks/data/training_atom_xavier_batchnorm/speech_commands_train/conv.ckpt-3100"
INFO:tensorflow:Step #3110: rate 0.000010, accuracy 93.0%, cross entropy 0.305975
INFO:tensorflow:Step #3120: rate 0.000010, accuracy 91.0%, cross entropy 0.290839
INFO:tensorflow:Step #3130: rate 0.000010, accuracy 91.0%, cross entropy 0.284596
INFO:tensorflow:Step #3140: rate 0.000010, accuracy 91.0%, cross entropy 0.264951
INFO:tensorflow:Step #3150: rate 0.000010, accuracy 91.0%, cross entropy 0.409717
INFO:tensorflow:Step #3160: rate 0.000010, accuracy 88.0%, cross ent

In [13]:
set_size = audio_processor.set_size('testing')
tf.logging.info('set_size=%d', set_size)
total_accuracy = 0
total_conf_matrix = None
for i in xrange(0, set_size, FLAGS.batch_size):
    test_fingerprints, test_ground_truth = audio_processor.get_data(
        FLAGS.batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
    test_accuracy, conf_matrix = sess.run(
        [evaluation_step, confusion_matrix],
        feed_dict={
            fingerprint_input: test_fingerprints,
            ground_truth_input: test_ground_truth,
            dropout_prob: 1.0
        })
    batch_size = min(FLAGS.batch_size, set_size - i)
    total_accuracy += (test_accuracy * batch_size) / set_size
    if total_conf_matrix is None:
        total_conf_matrix = conf_matrix
    else:
        total_conf_matrix += conf_matrix
tf.logging.info('Confusion Matrix:\n %s' % (total_conf_matrix))
tf.logging.info('Final test accuracy = %.1f%% (N=%d)' % (total_accuracy * 100,
                                                         set_size))

INFO:tensorflow:set_size=3081
INFO:tensorflow:Confusion Matrix:
 [[257   0   0   0   0   0   0   0   0   0   0   0]
 [  0 183   5   8   2   7   2   2  33   2   0  13]
 [  0   6 240   2   1   0   4   1   2   0   0   0]
 [  0   2   2 224   0   2   0   0   1   4   1  16]
 [  0   0   1   2 232   0   1   0   5  24   2   5]
 [  0   4   0  24   0 202   0   0   7   0   0  16]
 [  0   4  17   3   4   0 231   1   4   1   0   2]
 [  0  21   1   2   1   0   2 224   6   1   0   1]
 [  0   1   1   0   1   1   1   0 239   2   0   0]
 [  0   3   0   0   2   1   2   1  12 241   0   0]
 [  0   5   0   0   9   1   0   0   1   6 217  10]
 [  0   7   2  27   1   4   0   0   1   2   1 206]]
INFO:tensorflow:Final test accuracy = 87.5% (N=3081)


### Use the trained model to make a prediction

In [14]:
import csv

# use the model to predict
os.chdir (TEST_DIR)
print (os.getcwd())

file_names = os.listdir(TEST_DIR)
#file_names = map(lambda x: TEST_DIR + '/' + x, os.listdir(TEST_DIR))
#print (len(file_names))
#print (file_names[:10])
words_list = input_data.prepare_words_list(FLAGS.wanted_words.split(','))
#print (words_list)
set_size = len(file_names)
subm = zip(file_names, ('silence' for x in range(set_size)))
#print (len(subm))
#print (subm[:5])

# process the audio files
batch_num = 0
for i in xrange(0, set_size, FLAGS.batch_size):
    batch_size = min(FLAGS.batch_size, set_size - i)
    batch_filenames = file_names[i:i+batch_size]
    predict_fingerprints = audio_processor.get_predicion_data(
        batch_filenames, model_settings, sess)

    test_accuracy = sess.run(
        predicted_indices,
        feed_dict={
            fingerprint_input: predict_fingerprints,
            #ground_truth_input: test_ground_truth,
            dropout_prob: 1.0
        })
    
    '''
    if batch_num % 100 == 0:
        print ("i {}, batch num {}, batch size = {}".format(i, batch_num, batch_size))
        print (batch_filenames[10:20])
        print (test_accuracy[10:20])
    '''
    for j in range(batch_size):
        subm[i+j] = (subm[i+j][0],words_list[test_accuracy[j]])
    batch_num += 1



/notebooks/data/test/audio


In [15]:
for i in range(10,20):
    print (batch_filenames[i])
    print (words_list[test_accuracy[i]])
    print (subm[i])

clip_06786ab06.wav
_unknown_
('clip_3c0b101e0.wav', 'right')
clip_a629a0a8c.wav
_unknown_
('clip_24d52f235.wav', 'off')
clip_58b5acafb.wav
go
('clip_47cc3ea4e.wav', '_unknown_')
clip_c0e7e1556.wav
_unknown_
('clip_28ce90f4b.wav', '_unknown_')
clip_62e2c644e.wav
on
('clip_d83092849.wav', '_silence_')
clip_0768f2c6f.wav
_unknown_
('clip_798b8971d.wav', '_unknown_')
clip_1a9ad5f94.wav
off
('clip_9c432a821.wav', 'yes')
clip_592bb8eab.wav
_unknown_
('clip_c2e9fd918.wav', '_unknown_')
clip_bb779fa23.wav
go
('clip_67a34437e.wav', '_silence_')
clip_866c6ffb8.wav
_silence_
('clip_5ef10de58.wav', 'on')


In [16]:
test_accuracy

array([ 2,  0,  0,  9,  1,  0,  9, 10,  8,  2,  1,  1, 11,  1,  8,  1,  9,
        1, 11,  0,  1,  6, 11,  1,  1,  6,  8,  8,  1,  3,  0,  9,  2,  6,
        1,  4,  6,  1])

In [17]:
print (subm[:10])

[('clip_55e803b0c.wav', 'on'), ('clip_f248fa402.wav', '_silence_'), ('clip_04476ba6d.wav', '_unknown_'), ('clip_aa853b568.wav', '_unknown_'), ('clip_b8eb354f8.wav', '_unknown_'), ('clip_541afdf9b.wav', 'go'), ('clip_04fcbf5e4.wav', 'on'), ('clip_3ec4ed90b.wav', 'yes'), ('clip_4bc631523.wav', '_unknown_'), ('clip_41a2b5699.wav', '_unknown_')]


### Create submission file

In [18]:
# submission1.csv : sample submission
# submission2.csv : TensorFlow example code prediction, trained with Speech Commands Data Set v0.01
# submission3.csv : TensorFlow example code prediction, trained with Kaggle Data Set
# submission4.csv : TensorFlow example code prediction, trained using AtomOptimizer
# submission5.csv : TensorFlow example code prediction, using Xavier initialization
# submission6.csv : TensorFlow example code prediction, using batch normalization
submission_file_name = DATA_DIR+'/submission6.csv'
print (submission_file_name)
#np.savetxt(submission_file_name, subm, header='fname,label', comments='')
with open(submission_file_name, 'wb') as outfile:
    wr = csv.writer(outfile)
    wr.writerow(('fname', 'label'))
    for i in range(len(subm)):
        wr.writerow(subm[i])

/notebooks/data/submission6.csv


In [19]:
from IPython.display import FileLink
FileLink(submission_file_name)

/notebooks/data/submission6.csv